In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import random
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Set the paths to the dataset folders
plain_folder = '/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/plain'
roll_folder = '/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/roll'

# Set hyperparameters
input_shape = (256, 256, 1)
batch_size = 32
epochs = 10
learning_rate = 0.001

In [4]:
# Function to load and preprocess an image
def load_image(image_path):
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize(input_shape[:2])  # Resize to input shape
    image = np.array(image)  # Convert to numpy array
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    return image

In [16]:
# Generate pairs of plain and roll fingerprint images
def generate_pairs(plain_folder, roll_folder):
    plain_images = os.listdir(plain_folder)
    roll_images = os.listdir(roll_folder)
    pairs = []
    labels = []
    matchCount=0
    NotmatchCount=0

    for plain_image in plain_images:
        if(matchCount==1000):
          break
        plain_image_temp=plain_image.split('.')[0]
        plain_id = plain_image_temp.split('_')[0]
        end_id =plain_image_temp.split('_')[3]
        roll_image = [roll_image for roll_image in roll_images if roll_image.split('.')[0].startswith(plain_id) and roll_image.split('.')[0].endswith(end_id)]

        if len(roll_image) > 0:
            roll_image = random.choice(roll_image)
            plain_path = os.path.join(plain_folder, plain_image)
            roll_path = os.path.join(roll_folder, roll_image)
            pairs.append([plain_path, roll_path])
            labels.append(1)  # Positive pair
            matchCount=matchCount+1
    print(plain_id)
    print(end_id)
    for plain_image in plain_images:
        if(NotmatchCount==1000):
          break
        plain_image_temp=plain_image.split('.')[0]
        plain_id = plain_image_temp.split('_')[0]
        end_id =plain_image_temp.split('_')[3]
        impostor_roll_images = [roll_image for roll_image in roll_images if not roll_image.startswith(plain_id) and not roll_image.split('.')[0].endswith(end_id)]
        if len(impostor_roll_images) > 0:
            impostor_roll_image = random.choice(impostor_roll_images)
            plain_path = os.path.join(plain_folder, plain_image)
            impostor_roll_path = os.path.join(roll_folder, impostor_roll_image)
            pairs.append([plain_path, impostor_roll_path])
            labels.append(0)  # Negative pair
            NotmatchCount=NotmatchCount+1
    return pairs, labels

In [17]:
pairs, labels=generate_pairs(plain_folder, roll_folder)

00001514
04


In [18]:
count=0
for data, lab in zip(pairs, labels):
  if(count==10):
    break
  if(lab==1):
    print(data)
    print(lab)
    count=count+1

['/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/plain/00001823_plain_500_02.png', '/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/roll/00001823_roll_500_02.png']
1
['/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/plain/00001172_plain_500_02.png', '/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/roll/00001172_roll_500_02.png']
1
['/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/plain/00001542_plain_500_02.png', '/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/roll/00001542_roll_500_02.png']
1
['/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/plain/00001260_plain_500_04.png', '/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/roll/00001260_roll_500_04.png']
1
['/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/plain/00001039_plain_500_10.png', '/content/drive/MyDrive/NIST_pair_data/sd300a/images/500/png/roll/00001039_roll_500_10.png']
1
['/content/drive/MyDrive/NIST_pair_data/sd300

In [19]:
import random

def split_data(pairs, labels, train_ratio, test_ratio, val_ratio):
    # Calculate the number of samples for each set
    total_samples = len(pairs)
    train_samples = int(train_ratio * total_samples)
    test_samples = int(test_ratio * total_samples)
    val_samples = int(val_ratio * total_samples)

    # Combine pairs with labels
    data = list(zip(pairs, labels))
    # Shuffle the data randomly
    random.shuffle(data)

    # Split the data into train, test, and validation sets
    train_data = data[:train_samples]
    test_data = data[train_samples:train_samples + test_samples]
    val_data = data[train_samples + test_samples:train_samples + test_samples + val_samples]

    # Unpack the pairs and labels from the split sets
    train_pairs, train_labels = zip(*train_data)
    test_pairs, test_labels = zip(*test_data)
    val_pairs, val_labels = zip(*val_data)

    return train_pairs, train_labels, test_pairs, test_labels, val_pairs, val_labels

# Generate pairs and labels using the given method
pairs, labels = generate_pairs(plain_folder, roll_folder)

# Split the data into train, test, and validation sets
train_pairs, train_labels, test_pairs, test_labels, val_pairs, val_labels = split_data(pairs, labels, 0.7, 0.2, 0.1)

# Print the number of samples in each set
print("Train samples:", len(train_pairs))
print("Test samples:", len(test_pairs))
print("Validation samples:", len(val_pairs))


00001514
04
Train samples: 1400
Test samples: 400
Validation samples: 200


In [20]:
from tensorflow.keras.layers import Input, multiply, Lambda, add, concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import random
import string
import math

In [21]:
# Create the Siamese network architecture
def create_siamese_model():
    input_1 = layers.Input(input_shape)
    input_2 = layers.Input(input_shape)

    base_network = models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
    ])

    feature_1 = base_network(input_1)
    feature_2 = base_network(input_2)

    # Measure similarity between the two feature vectors
    similarity = layers.Dot(axes=1, normalize=True)([feature_1, feature_2])

    siamese_model = models.Model(inputs=[input_1, input_2], outputs=similarity)
    return siamese_model

In [22]:
import os
import random
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

In [23]:
# Load and preprocess the image pairs
pairs, labels = generate_pairs(plain_folder, roll_folder)

# Shuffle the pairs and labels
combined = list(zip(pairs, labels))
random.shuffle(combined)
pairs, labels = zip(*combined)

# Split the data into training and validation sets
split = int(0.8 * len(pairs))  # 80% for training, 20% for validation
train_pairs, val_pairs = pairs[:split], pairs[split:]
train_labels, val_labels = labels[:split], labels[split:]

import tensorflow as tf

tf.config.run_functions_eagerly(True)

# Create Siamese model
siamese_model = create_siamese_model()
siamese_model.compile(optimizer=optimizers.Adam(learning_rate), loss='binary_crossentropy', run_eagerly=True)



00001514
04


In [24]:
def data_generator(image_pairs, labels, batch_size):
    while True:
        batch_indices = np.random.choice(len(image_pairs), size=batch_size, replace=False)
        batch_pairs = [image_pairs[i] for i in batch_indices]
        batch_labels = [labels[i] for i in batch_indices]

        x1 = []
        x2 = []
        for pair in batch_pairs:
            image_1 = load_image(pair[0])
            image_2 = load_image(pair[1])
            x1.append(image_1)
            x2.append(image_2)

        x1 = np.array(x1)
        x2 = np.array(x2)
        y = np.array(batch_labels)

        yield [x1, x2], y

In [25]:
train_generator = data_generator(train_pairs, train_labels, batch_size)
val_generator = data_generator(val_pairs, val_labels, batch_size)

# Train the Siamese model
steps_per_epoch = len(train_pairs) // batch_size
validation_steps = len(val_pairs) // batch_size

siamese_model.fit(train_generator,
                  steps_per_epoch=steps_per_epoch,
                  epochs=epochs,
                  validation_data=val_generator,
                  validation_steps=validation_steps)

# Save the trained model
siamese_model.save('siamese_model.h5')

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


50/50 [==============================] - 1396s 28s/step - loss: 1.9042 - val_loss: 0.7245
Epoch 2/10
50/50 [==============================] - 502s 10s/step - loss: 0.5865 - val_loss: 0.7556
Epoch 3/10
50/50 [==============================] - 170s 3s/step - loss: 0.4530 - val_loss: 0.6828
Epoch 4/10
50/50 [==============================] - 94s 2s/step - loss: 0.3611 - val_loss: 1.0923
Epoch 5/10
50/50 [==============================] - 54s 1s/step - loss: 0.6043 - val_loss: 2.1795
Epoch 6/10
50/50 [==============================] - 46s 929ms/step - loss: 1.4151 - val_loss: 0.9921
Epoch 7/10
50/50 [==============================] - 43s 881ms/step - loss: 0.7485 - val_loss: 0.7304
Epoch 8/10
50/50 [==============================] - 44s 901ms/step - loss: 0.5172 - val_loss: 0.7644
Epoch 9/10
50/50 [==============================] - 43s 881ms/step - loss: 0.4249 - val_loss: 0.7306
Epoch 10/10
50/50 [==============================] - 44s 886ms/step - loss: 0.3178 - val_loss: 0.8804


In [26]:
import shutil
# Copy the file
shutil.copyfile('/content/siamese_model.h5', '/content/drive/MyDrive/nikita/siamese_model.h5')

print('File copied successfully.')

File copied successfully.


In [27]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import confusion_matrix, accuracy_score

# Load the trained Siamese model
siamese_model = load_model('/content/drive/MyDrive/nikita/siamese_model.h5')

# Initialize lists to store predicted labels and ground truth labels
predicted_labels = []
true_labels = []

# Iterate over the test dataset
for pair, label in zip(test_pairs, test_labels):
    # Load and preprocess the images (similar to previous examples)
    img1 = load_img(pair[0], color_mode='grayscale', target_size=(256, 256))
    img2 = load_img(pair[1], color_mode='grayscale', target_size=(256, 256))
    img1 = img_to_array(img1) / 255.0
    img2 = img_to_array(img2) / 255.0
    X_test_1 = np.expand_dims(img1, axis=0)
    X_test_2 = np.expand_dims(img2, axis=0)

    # Pass the test pair through the Siamese model and get the predicted similarity score
    prediction = siamese_model.predict([X_test_1, X_test_2])

    # Assign a predicted label based on the similarity score (e.g., using a threshold)
    predicted_label = 1 if prediction >= 0.8 else 0

    # Store the predicted and ground truth labels
    predicted_labels.append(predicted_label)
    true_labels.append(label)

# Convert the lists to NumPy arrays
predicted_labels = np.array(predicted_labels)
true_labels = np.array(true_labels)

# Calculate the confusion matrix
confusion_mat = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(confusion_mat)

# Calculate the test accuracy
test_accuracy = accuracy_score(true_labels, predicted_labels)
print("Test Accuracy:", test_accuracy)

1/1 [==============================] - 0s 29ms/step
Confusion Matrix:
[[175  46]
 [ 55 124]]
Test Accuracy: 0.7475
